In [322]:
len(down_test_

SyntaxError: unexpected EOF while parsing (<ipython-input-322-04a515899fd7>, line 1)

In [323]:
len(down_test)

18

In [324]:
len(up_test)

12

In [325]:
len(up_train)

19

In [327]:
stock_train = get_stock_data('2016_stock_data.csv', '大立光')
stock2018 = get_stock_data('2017_stock_data.csv', '大立光')
# stock2018 = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
# stock_train = np.concatenate((stock2016, stock2017), axis=0)
end_value_train = conv_num(stock_train[:, 5])
end_value2018 = conv_num(stock2018[:, 5])
stock_date_train = stock_train[:, 1]
stock_date2018 = stock2018[:, 1]


In [328]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = get_doc_by_date(up_date_train, news)
down_docs_train = get_doc_by_date(down_date_train, news)


In [329]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = get_doc_by_date(up_date_test, news)
down_docs_test = get_doc_by_date(down_date_test, news)

# Request 3 - Total evaluation (find up down factor)

In [330]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


(2051, 1500)


In [331]:
test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
X_test = test_x.toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


(1922, 1500)


In [332]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[ 149 1221]
 [  63  489]]
             precision    recall  f1-score   support

          0       0.70      0.11      0.19      1370
          1       0.29      0.89      0.43       552

avg / total       0.58      0.33      0.26      1922

0.331945889698231


In [333]:
stopwords = []
with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
    for line in f1:
        stopwords = line.split()
stopwords[-1] = ' '

def get_company(company, stock_np, name_index = 0):
    indexs = []
    stock_np = np.array(stock_np)
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    return stock_np[indexs,:]

def get_stock_data(file, company_name = '大立光'):
    stock_data = pd.read_csv(file, encoding='UTF-8')
    stock_data_np = stock_data.values
    stock = get_company(company_name, stock_data_np)
    return stock[::-1,:]

def conv_num(nums):
    for i in range(len(nums)):
        nums[i] = nums[i].replace(',', '')
        nums[i] = float(nums[i])
    return nums

def conv_date(date):
    # date = np.array(date)
    for i in range(len(date)):
        d = re.findall('[0-9]+/[0-9]+/[0-9]+', date[i])[0]
        date[i] = datetime.datetime.strptime(date[i],'%Y/%M/%d').date()
        return date
    

def text_clean(text):
    text = text.lower()
    text = re.sub('<br>', ' ', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[^\w\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

def get_up_down_index(values, n = 5, r = 0.1):
    up = []
    down = []
	#後三天股價-原股價/原股價 ->漲跌幅度超過5%者
    for i in range(len(values) - n):
        dis  = values[i + n] - values[i]
        if abs(dis)/ values[i] >= r:
            up.append(i) if dis > 0 else down.append(i)
    return up, down

def get_doc_by_date(dates, data, keyWord, date_index = 4, title_index = 5, content_index = 7):
    docs = []
    for r in data:
        if r[date_index][:-9] in dates:
            for i in range(len(keyWord)):
                if keyWord[i] in r[content_index]:
                    temp = '%s %s %s' %(r[title_index], r[title_index], r[content_index])
                    docs.append(text_clean(temp))
    return docs

def tfidf_tool(corpus):
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords, max_features = 1500, max_df=0.8)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def get_top_feature(X, vectorizer):
    m = X.mean(axis=0).getA().reshape(-1)
    max_index = np.argsort(m)[::-1] 
    tokens = np.array(vectorizer.get_feature_names())
    return tokens[max_index]

def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def chi2(all_vectors, clas_vectors):
    expected = all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    signs = np.where(clas_value-expected, 2)/expected
    chi2_value[signs] *= -1
    return chi2_value

def find_week_month_cross (values, n=3, r = 0.03):
    result_true = []
    result_false = []
    pre_month = values[:20].mean()
    pre_week = values[15:20].mean()
    month = []
    week = []
    for i in range (21,len (values)):
        month_mov = values[i-20:i].mean()
        week_mov = values[i-5:i].mean()  
        month.append(month_mov)
        week.append(week_mov)
        if pre_month > pre_week and month_mov <= week_mov:
            if abs(values[i-n]-values[i])/values[i-n] > r:
                result_true.append (i-n)
        elif pre_month < pre_week and month_mov >= week_mov:
            if abs (values [i-n]-values[i])/values[i-n] > r:
               result_true.append(i-n)
        elif abs (week_mov-month_mov)/min(week_mov, month_mov) < 0.01:
            result_false.append (i-n)
        pre_month = month_mov
        pre_week = week_mov
    return result_true, result_false



# Requirement 1 - key word search

In [334]:
stock_train = get_stock_data('2016_stock_data.csv', '大立光')
stock2018 = get_stock_data('2017_stock_data.csv', '大立光')
# stock2018 = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
# stock_train = np.concatenate((stock2016, stock2017), axis=0)
end_value_train = conv_num(stock_train[:, 5])
end_value2018 = conv_num(stock2018[:, 5])
stock_date_train = stock_train[:, 1]
stock_date2018 = stock2018[:, 1]


KeyboardInterrupt: 

In [335]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = get_doc_by_date(up_date_train, news)
down_docs_train = get_doc_by_date(down_date_train, news)


TypeError: get_doc_by_date() missing 1 required positional argument: 'keyWord'

In [336]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = get_doc_by_date(up_date_test, news)
down_docs_test = get_doc_by_date(down_date_test, news)

TypeError: get_doc_by_date() missing 1 required positional argument: 'keyWord'

# Request 3 - Total evaluation (find up down factor)

In [337]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


KeyboardInterrupt: 

In [338]:
test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
X_test = test_x.toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


KeyboardInterrupt: 

In [339]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[ 149 1221]
 [  63  489]]
             precision    recall  f1-score   support

          0       0.70      0.11      0.19      1370
          1       0.29      0.89      0.43       552

avg / total       0.58      0.33      0.26      1922

0.331945889698231


# Requirement 1 - key word search

In [340]:
stock_train = get_stock_data('2016_stock_data.csv', '大立光')
stock2018 = get_stock_data('2017_stock_data.csv', '大立光')
# stock2018 = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
# stock_train = np.concatenate((stock2016, stock2017), axis=0)
end_value_train = conv_num(stock_train[:, 5])
end_value2018 = conv_num(stock2018[:, 5])
stock_date_train = stock_train[:, 1]
stock_date2018 = stock2018[:, 1]


In [341]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = get_doc_by_date(up_date_train, news, ['大立光', '漲', '股票'])
down_docs_train = get_doc_by_date(down_date_train, news, ['大立光', '跌', '股票'])


TypeError: argument of type 'float' is not iterable

In [342]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = get_doc_by_date(up_date_test, news, ['大立光', '漲', '股票'])
down_docs_test = get_doc_by_date(down_date_test, news, ['大立光', '跌', '股票'])

TypeError: argument of type 'float' is not iterable

# Request 3 - Total evaluation (find up down factor)

In [343]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


KeyboardInterrupt: 

In [344]:
test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
X_test = test_x.toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)



KeyboardInterrupt



In [345]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[ 149 1221]
 [  63  489]]
             precision    recall  f1-score   support

          0       0.70      0.11      0.19      1370
          1       0.29      0.89      0.43       552

avg / total       0.58      0.33      0.26      1922

0.331945889698231


In [346]:
stopwords = []
with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
    for line in f1:
        stopwords = line.split()
stopwords[-1] = ' '

def get_company(company, stock_np, name_index = 0):
    indexs = []
    stock_np = np.array(stock_np)
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    return stock_np[indexs,:]

def get_stock_data(file, company_name = '大立光'):
    stock_data = pd.read_csv(file, encoding='UTF-8')
    stock_data_np = stock_data.values
    stock = get_company(company_name, stock_data_np)
    return stock[::-1,:]

def conv_num(nums):
    for i in range(len(nums)):
        nums[i] = nums[i].replace(',', '')
        nums[i] = float(nums[i])
    return nums

def conv_date(date):
    # date = np.array(date)
    for i in range(len(date)):
        d = re.findall('[0-9]+/[0-9]+/[0-9]+', date[i])[0]
        date[i] = datetime.datetime.strptime(date[i],'%Y/%M/%d').date()
        return date
    

def text_clean(text):
    text = text.lower()
    text = re.sub('<br>', ' ', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[^\w\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

def get_up_down_index(values, n = 5, r = 0.1):
    up = []
    down = []
	#後三天股價-原股價/原股價 ->漲跌幅度超過5%者
    for i in range(len(values) - n):
        dis  = values[i + n] - values[i]
        if abs(dis)/ values[i] >= r:
            up.append(i) if dis > 0 else down.append(i)
    return up, down

def get_doc_by_date(dates, data, keyWord, date_index = 4, title_index = 5, content_index = 7):
    docs = []
    for r in data:
        if r[date_index][:-9] in dates:
            for i in range(len(keyWord)):
                print(keyWord[i], type(keyWord[i]))
                if keyWord[i] in r[content_index]:
                    temp = '%s %s %s' %(r[title_index], r[title_index], r[content_index])
                    docs.append(text_clean(temp))
    return docs

def tfidf_tool(corpus):
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords, max_features = 1500, max_df=0.8)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def get_top_feature(X, vectorizer):
    m = X.mean(axis=0).getA().reshape(-1)
    max_index = np.argsort(m)[::-1] 
    tokens = np.array(vectorizer.get_feature_names())
    return tokens[max_index]

def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def chi2(all_vectors, clas_vectors):
    expected = all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    signs = np.where(clas_value-expected, 2)/expected
    chi2_value[signs] *= -1
    return chi2_value

def find_week_month_cross (values, n=3, r = 0.03):
    result_true = []
    result_false = []
    pre_month = values[:20].mean()
    pre_week = values[15:20].mean()
    month = []
    week = []
    for i in range (21,len (values)):
        month_mov = values[i-20:i].mean()
        week_mov = values[i-5:i].mean()  
        month.append(month_mov)
        week.append(week_mov)
        if pre_month > pre_week and month_mov <= week_mov:
            if abs(values[i-n]-values[i])/values[i-n] > r:
                result_true.append (i-n)
        elif pre_month < pre_week and month_mov >= week_mov:
            if abs (values [i-n]-values[i])/values[i-n] > r:
               result_true.append(i-n)
        elif abs (week_mov-month_mov)/min(week_mov, month_mov) < 0.01:
            result_false.append (i-n)
        pre_month = month_mov
        pre_week = week_mov
    return result_true, result_false



In [347]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = get_doc_by_date(up_date_train, news, ['大立光', '漲', '股票'])
down_docs_train = get_doc_by_date(down_date_train, news, ['大立光', '跌', '股票'])


大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str'>
股票 <class 'str'>
大立光 <class 'str'>
漲 <class 'str

TypeError: argument of type 'float' is not iterable

In [348]:
stopwords = []
with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
    for line in f1:
        stopwords = line.split()
stopwords[-1] = ' '

def get_company(company, stock_np, name_index = 0):
    indexs = []
    stock_np = np.array(stock_np)
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    return stock_np[indexs,:]

def get_stock_data(file, company_name = '大立光'):
    stock_data = pd.read_csv(file, encoding='UTF-8')
    stock_data_np = stock_data.values
    stock = get_company(company_name, stock_data_np)
    return stock[::-1,:]

def conv_num(nums):
    for i in range(len(nums)):
        nums[i] = nums[i].replace(',', '')
        nums[i] = float(nums[i])
    return nums

def conv_date(date):
    # date = np.array(date)
    for i in range(len(date)):
        d = re.findall('[0-9]+/[0-9]+/[0-9]+', date[i])[0]
        date[i] = datetime.datetime.strptime(date[i],'%Y/%M/%d').date()
        return date
    

def text_clean(text):
    text = text.lower()
    text = re.sub('<br>', ' ', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[^\w\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

def get_up_down_index(values, n = 5, r = 0.1):
    up = []
    down = []
	#後三天股價-原股價/原股價 ->漲跌幅度超過5%者
    for i in range(len(values) - n):
        dis  = values[i + n] - values[i]
        if abs(dis)/ values[i] >= r:
            up.append(i) if dis > 0 else down.append(i)
    return up, down

def get_doc_by_date(dates, data, keyWord, date_index = 4, title_index = 5, content_index = 7):
    docs = []
    for r in data:
        if r[date_index][:-9] in dates:
            for i in range(len(keyWord)):
                print(r[content_index], type(r[content_index]))
                if keyWord[i] in r[content_index]:
                    temp = '%s %s %s' %(r[title_index], r[title_index], r[content_index])
                    docs.append(text_clean(temp))
                    break
    return docs

def tfidf_tool(corpus):
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords, max_features = 1500, max_df=0.8)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def get_top_feature(X, vectorizer):
    m = X.mean(axis=0).getA().reshape(-1)
    max_index = np.argsort(m)[::-1] 
    tokens = np.array(vectorizer.get_feature_names())
    return tokens[max_index]

def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def chi2(all_vectors, clas_vectors):
    expected = all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    signs = np.where(clas_value-expected, 2)/expected
    chi2_value[signs] *= -1
    return chi2_value

def find_week_month_cross (values, n=3, r = 0.03):
    result_true = []
    result_false = []
    pre_month = values[:20].mean()
    pre_week = values[15:20].mean()
    month = []
    week = []
    for i in range (21,len (values)):
        month_mov = values[i-20:i].mean()
        week_mov = values[i-5:i].mean()  
        month.append(month_mov)
        week.append(week_mov)
        if pre_month > pre_week and month_mov <= week_mov:
            if abs(values[i-n]-values[i])/values[i-n] > r:
                result_true.append (i-n)
        elif pre_month < pre_week and month_mov >= week_mov:
            if abs (values [i-n]-values[i])/values[i-n] > r:
               result_true.append(i-n)
        elif abs (week_mov-month_mov)/min(week_mov, month_mov) < 0.01:
            result_false.append (i-n)
        pre_month = month_mov
        pre_week = week_mov
    return result_true, result_false



In [349]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = get_doc_by_date(up_date_train, news, ['大立光', '漲', '股票'])
down_docs_train = get_doc_by_date(down_date_train, news, ['大立光', '跌', '股票'])


泛歐FTEurofirst 300指數和績優股Euro STOXX指數分別上揚0.5%和0.9%。泛歐FTEurofirst 300指數上週重挫7%。<BR> 中國大陸股市暴跌引爆股災，蔓延至全球股市。中央社（翻譯） <class 'str'>
泛歐FTEurofirst 300指數和績優股Euro STOXX指數分別上揚0.5%和0.9%。泛歐FTEurofirst 300指數上週重挫7%。<BR> 中國大陸股市暴跌引爆股災，蔓延至全球股市。中央社（翻譯） <class 'str'>
泛歐FTEurofirst 300指數和績優股Euro STOXX指數分別上揚0.5%和0.9%。泛歐FTEurofirst 300指數上週重挫7%。<BR> 中國大陸股市暴跌引爆股災，蔓延至全球股市。中央社（翻譯） <class 'str'>
（法新社底特律11日電） 在「汽車與卡車雜誌」（Car & Truck Magazine）今天舉行的頒獎典禮上，本田Civic（Honda Civic）和富豪休旅車XC 90（Volvo XC90）分別膺選北美年度風雲汽車和風雲卡車。<BR> 美國富豪汽車（Volvo Cars USA）執行長柯西梅克斯（Lex Kerrsemakers）表示，由53名汽車記者遴選的此獎項，將有助於放眼重新躋身美國更大汽車品牌的富豪汽車。富豪隸屬中國大陸吉利汽車集團（Geely）。<BR> 柯西梅克斯說，此款車輛結合富豪的超級安全信譽和更佳油耗，部分是拜採用4缸引擎之賜，「這顯示我們顯然走在正確途徑上」。<BR> 柯西梅克斯指出，中國大陸母公司賦予富豪獨立營運地位。<BR> 本田執行副總裁蒙德爾（John Mendel）表示，他對獲獎深感振奮，並稱「本田Civic不會止步於此」。 <class 'str'>
（法新社底特律11日電） 在「汽車與卡車雜誌」（Car & Truck Magazine）今天舉行的頒獎典禮上，本田Civic（Honda Civic）和富豪休旅車XC 90（Volvo XC90）分別膺選北美年度風雲汽車和風雲卡車。<BR> 美國富豪汽車（Volvo Cars USA）執行長柯西梅克斯（Lex Kerrsemakers）表示，由53名汽車記者遴選的此獎項，將有助於放眼重新躋身美國更大汽車品牌的富豪汽車。富豪隸屬中國大陸吉利汽車集團（Gee

TypeError: argument of type 'float' is not iterable

In [350]:
stopwords = []
with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
    for line in f1:
        stopwords = line.split()
stopwords[-1] = ' '

def get_company(company, stock_np, name_index = 0):
    indexs = []
    stock_np = np.array(stock_np)
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    return stock_np[indexs,:]

def get_stock_data(file, company_name = '大立光'):
    stock_data = pd.read_csv(file, encoding='UTF-8')
    stock_data_np = stock_data.values
    stock = get_company(company_name, stock_data_np)
    return stock[::-1,:]

def conv_num(nums):
    for i in range(len(nums)):
        nums[i] = nums[i].replace(',', '')
        nums[i] = float(nums[i])
    return nums

def conv_date(date):
    # date = np.array(date)
    for i in range(len(date)):
        d = re.findall('[0-9]+/[0-9]+/[0-9]+', date[i])[0]
        date[i] = datetime.datetime.strptime(date[i],'%Y/%M/%d').date()
        return date
    

def text_clean(text):
    text = text.lower()
    text = re.sub('<br>', ' ', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[^\w\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

def get_up_down_index(values, n = 5, r = 0.1):
    up = []
    down = []
	#後三天股價-原股價/原股價 ->漲跌幅度超過5%者
    for i in range(len(values) - n):
        dis  = values[i + n] - values[i]
        if abs(dis)/ values[i] >= r:
            up.append(i) if dis > 0 else down.append(i)
    return up, down

def get_doc_by_date(dates, data, keyWord, date_index = 4, title_index = 5, content_index = 7):
    docs = []
    for r in data:
        if r[date_index][:-9] in dates:
            for i in range(len(keyWord)):
                if keyWord[i] in r[content_index]:
                    print(r[content_index], type(r[content_index]))
                    temp = '%s %s %s' %(r[title_index], r[title_index], r[content_index])
                    docs.append(text_clean(temp))
                    break
    return docs

def tfidf_tool(corpus):
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords, max_features = 1500, max_df=0.8)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def get_top_feature(X, vectorizer):
    m = X.mean(axis=0).getA().reshape(-1)
    max_index = np.argsort(m)[::-1] 
    tokens = np.array(vectorizer.get_feature_names())
    return tokens[max_index]

def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def chi2(all_vectors, clas_vectors):
    expected = all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    signs = np.where(clas_value-expected, 2)/expected
    chi2_value[signs] *= -1
    return chi2_value

def find_week_month_cross (values, n=3, r = 0.03):
    result_true = []
    result_false = []
    pre_month = values[:20].mean()
    pre_week = values[15:20].mean()
    month = []
    week = []
    for i in range (21,len (values)):
        month_mov = values[i-20:i].mean()
        week_mov = values[i-5:i].mean()  
        month.append(month_mov)
        week.append(week_mov)
        if pre_month > pre_week and month_mov <= week_mov:
            if abs(values[i-n]-values[i])/values[i-n] > r:
                result_true.append (i-n)
        elif pre_month < pre_week and month_mov >= week_mov:
            if abs (values [i-n]-values[i])/values[i-n] > r:
               result_true.append(i-n)
        elif abs (week_mov-month_mov)/min(week_mov, month_mov) < 0.01:
            result_false.append (i-n)
        pre_month = month_mov
        pre_week = week_mov
    return result_true, result_false



In [351]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = get_doc_by_date(up_date_train, news, ['大立光', '漲', '股票'])
down_docs_train = get_doc_by_date(down_date_train, news, ['大立光', '跌', '股票'])


鉅亨網編譯郭照青低殖利率的歐元兌美元週一連續第二天下跌，日元六天來首度走低。全球股市回穩，促使投資人出脫避險貨幣，尋求較高的收益率。<BR>華爾街今日走勢震盪，但收復了部份開年以來的跌幅，削弱了歐元與日元的吸引力。<BR>近來利差交易中，投資人借入低利率的歐元與日元，買進其他高收益率貨幣。<BR>分析師說，風險氣氛溫和改善，使得歐元兌美元下跌。<BR>但歐元回檔可能僅屬短暫，因為中國的不確定性及商品價格下跌，仍為投資人擔憂。<BR>尾盤交易中，歐元下跌0.5%至1.0873美元，下跌0.4%至127.73日元。<BR>美元指數上漲0.59，或0.60%，至98.86。<BR>目前歐元兌美元的漲勢是否會持續，有待觀察。長期低點是否已出現，亦不確定。<BR>風險氣氛改善，有助支撐美元。最近交易中，美元上漲0.2%至117.47日元。<BR>美元上漲0.5%至0.9997瑞士法郎。<BR>中國人民幣週一上漲，兌美元創四個月來最大單日漲幅，報導指出，北京再度積極干預，支撐了人民幣上漲。<BR>人民幣上漲0.35%至6.5701兌1美元。離岸匯率上漲1.2%至6.6030兌1美元。 <class 'str'>
（中央社台北2016年1月12日電）美股收盤前最後1小時反彈，此時投資人正等候財報季來臨，油價則跌破每桶32美元，為12年來首見。<BR>道瓊工業指數上揚52.12點或0.32%，以16398.57點作收；標準普爾指數小漲1.64點或0.09%，收在1923.67點。<BR>儘管尾盤上拉，科技股那斯達克指數仍未守住，終場下跌5.64點或0.12%，收4637.99點。（譯者：中央社張佑之） <class 'str'>
(中央社台北2016年 1月12日電)友達光電在NYSE掛牌ADR 以2.69美元作收，上漲0.01美元，漲幅0.37%。<BR>由於每單位友達 ADR相當於台股10股友達普通股，以 1月11日新台幣匯市收盤價33.651元計算，相當台股友達每股為 9.05元。<BR>最近10個交易日行情如下：<BR> 友達 ADR 近十個交易日行情 交易日收盤價漲跌漲跌幅%匯率折台股01/112.690.010.3733.6519.0501/082.680.031.1333.5018.9801/072.65-0.04-1.4933.5308.8901/062.69-0.

TypeError: argument of type 'float' is not iterable

In [352]:
stopwords = []
with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
    for line in f1:
        stopwords = line.split()
stopwords[-1] = ' '

def get_company(company, stock_np, name_index = 0):
    indexs = []
    stock_np = np.array(stock_np)
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    return stock_np[indexs,:]

def get_stock_data(file, company_name = '大立光'):
    stock_data = pd.read_csv(file, encoding='UTF-8')
    stock_data_np = stock_data.values
    stock = get_company(company_name, stock_data_np)
    return stock[::-1,:]

def conv_num(nums):
    for i in range(len(nums)):
        nums[i] = nums[i].replace(',', '')
        nums[i] = float(nums[i])
    return nums

def conv_date(date):
    # date = np.array(date)
    for i in range(len(date)):
        d = re.findall('[0-9]+/[0-9]+/[0-9]+', date[i])[0]
        date[i] = datetime.datetime.strptime(date[i],'%Y/%M/%d').date()
        return date
    

def text_clean(text):
    text = text.lower()
    text = re.sub('<br>', ' ', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[^\w\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

def get_up_down_index(values, n = 5, r = 0.1):
    up = []
    down = []
	#後三天股價-原股價/原股價 ->漲跌幅度超過5%者
    for i in range(len(values) - n):
        dis  = values[i + n] - values[i]
        if abs(dis)/ values[i] >= r:
            up.append(i) if dis > 0 else down.append(i)
    return up, down

def get_doc_by_date(dates, data, date_index = 4, title_index = 5, content_index = 7):
    docs = []
    for r in data:
        if r[date_index][:-9] in dates:
            temp = '%s %s %s' %(r[title_index], r[title_index], r[content_index])
            docs.append(text_clean(temp))
    return docs

def tfidf_tool(corpus):
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords, max_features = 1500, max_df=0.8)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def get_top_feature(X, vectorizer):
    m = X.mean(axis=0).getA().reshape(-1)
    max_index = np.argsort(m)[::-1] 
    tokens = np.array(vectorizer.get_feature_names())
    return tokens[max_index]

def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def chi2(all_vectors, clas_vectors):
    expected = all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    signs = np.where(clas_value-expected, 2)/expected
    chi2_value[signs] *= -1
    return chi2_value

def find_week_month_cross (values, n=3, r = 0.03):
    result_true = []
    result_false = []
    pre_month = values[:20].mean()
    pre_week = values[15:20].mean()
    month = []
    week = []
    for i in range (21,len (values)):
        month_mov = values[i-20:i].mean()
        week_mov = values[i-5:i].mean()  
        month.append(month_mov)
        week.append(week_mov)
        if pre_month > pre_week and month_mov <= week_mov:
            if abs(values[i-n]-values[i])/values[i-n] > r:
                result_true.append (i-n)
        elif pre_month < pre_week and month_mov >= week_mov:
            if abs (values [i-n]-values[i])/values[i-n] > r:
               result_true.append(i-n)
        elif abs (week_mov-month_mov)/min(week_mov, month_mov) < 0.01:
            result_false.append (i-n)
        pre_month = month_mov
        pre_week = week_mov
    return result_true, result_false



In [353]:
stopwords = []
with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
    for line in f1:
        stopwords = line.split()
stopwords[-1] = ' '

def get_company(company, stock_np, name_index = 0):
    indexs = []
    stock_np = np.array(stock_np)
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    return stock_np[indexs,:]

def get_stock_data(file, company_name = '大立光'):
    stock_data = pd.read_csv(file, encoding='UTF-8')
    stock_data_np = stock_data.values
    stock = get_company(company_name, stock_data_np)
    return stock[::-1,:]

def conv_num(nums):
    for i in range(len(nums)):
        nums[i] = nums[i].replace(',', '')
        nums[i] = float(nums[i])
    return nums

def conv_date(date):
    # date = np.array(date)
    for i in range(len(date)):
        d = re.findall('[0-9]+/[0-9]+/[0-9]+', date[i])[0]
        date[i] = datetime.datetime.strptime(date[i],'%Y/%M/%d').date()
        return date
    

def text_clean(text):
    text = text.lower()
    text = re.sub('<br>', ' ', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[^\w\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

def get_up_down_index(values, n = 5, r = 0.1):
    up = []
    down = []
	#後三天股價-原股價/原股價 ->漲跌幅度超過5%者
    for i in range(len(values) - n):
        dis  = values[i + n] - values[i]
        if abs(dis)/ values[i] >= r:
            up.append(i) if dis > 0 else down.append(i)
    return up, down

def get_doc_by_date(dates, data, date_index = 4, title_index = 5, content_index = 7):
    docs = []
    for r in data:
        if r[date_index][:-9] in dates:
            temp = '%s %s %s' %(r[title_index], r[title_index], r[content_index])
            docs.append(text_clean(temp))
    return docs

def keyword_select(docs, kewords):
    selected = []
    for text in docs
        for i in range(len(kewords)):
            if kewords[i] in text:
                selected.append(text)
                break
    return selected

def tfidf_tool(corpus):
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords, max_features = 1500, max_df=0.8)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def get_top_feature(X, vectorizer):
    m = X.mean(axis=0).getA().reshape(-1)
    max_index = np.argsort(m)[::-1] 
    tokens = np.array(vectorizer.get_feature_names())
    return tokens[max_index]

def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def chi2(all_vectors, clas_vectors):
    expected = all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    signs = np.where(clas_value-expected, 2)/expected
    chi2_value[signs] *= -1
    return chi2_value

def find_week_month_cross (values, n=3, r = 0.03):
    result_true = []
    result_false = []
    pre_month = values[:20].mean()
    pre_week = values[15:20].mean()
    month = []
    week = []
    for i in range (21,len (values)):
        month_mov = values[i-20:i].mean()
        week_mov = values[i-5:i].mean()  
        month.append(month_mov)
        week.append(week_mov)
        if pre_month > pre_week and month_mov <= week_mov:
            if abs(values[i-n]-values[i])/values[i-n] > r:
                result_true.append (i-n)
        elif pre_month < pre_week and month_mov >= week_mov:
            if abs (values [i-n]-values[i])/values[i-n] > r:
               result_true.append(i-n)
        elif abs (week_mov-month_mov)/min(week_mov, month_mov) < 0.01:
            result_false.append (i-n)
        pre_month = month_mov
        pre_week = week_mov
    return result_true, result_false



SyntaxError: invalid syntax (<ipython-input-353-71b3fc883afa>, line 64)

In [354]:
stopwords = []
with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
    for line in f1:
        stopwords = line.split()
stopwords[-1] = ' '

def get_company(company, stock_np, name_index = 0):
    indexs = []
    stock_np = np.array(stock_np)
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    return stock_np[indexs,:]

def get_stock_data(file, company_name = '大立光'):
    stock_data = pd.read_csv(file, encoding='UTF-8')
    stock_data_np = stock_data.values
    stock = get_company(company_name, stock_data_np)
    return stock[::-1,:]

def conv_num(nums):
    for i in range(len(nums)):
        nums[i] = nums[i].replace(',', '')
        nums[i] = float(nums[i])
    return nums

def conv_date(date):
    # date = np.array(date)
    for i in range(len(date)):
        d = re.findall('[0-9]+/[0-9]+/[0-9]+', date[i])[0]
        date[i] = datetime.datetime.strptime(date[i],'%Y/%M/%d').date()
        return date
    

def text_clean(text):
    text = text.lower()
    text = re.sub('<br>', ' ', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[^\w\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

def get_up_down_index(values, n = 5, r = 0.1):
    up = []
    down = []
	#後三天股價-原股價/原股價 ->漲跌幅度超過5%者
    for i in range(len(values) - n):
        dis  = values[i + n] - values[i]
        if abs(dis)/ values[i] >= r:
            up.append(i) if dis > 0 else down.append(i)
    return up, down

def get_doc_by_date(dates, data, date_index = 4, title_index = 5, content_index = 7):
    docs = []
    for r in data:
        if r[date_index][:-9] in dates:
            temp = '%s %s %s' %(r[title_index], r[title_index], r[content_index])
            docs.append(text_clean(temp))
    return docs

def keyword_select(docs, kewords):
    selected = []
    for text in docs:
        for i in range(len(kewords)):
            if kewords[i] in text:
                selected.append(text)
                break
    return selected

def tfidf_tool(corpus):
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords, max_features = 1500, max_df=0.8)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def get_top_feature(X, vectorizer):
    m = X.mean(axis=0).getA().reshape(-1)
    max_index = np.argsort(m)[::-1] 
    tokens = np.array(vectorizer.get_feature_names())
    return tokens[max_index]

def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def chi2(all_vectors, clas_vectors):
    expected = all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    signs = np.where(clas_value-expected, 2)/expected
    chi2_value[signs] *= -1
    return chi2_value

def find_week_month_cross (values, n=3, r = 0.03):
    result_true = []
    result_false = []
    pre_month = values[:20].mean()
    pre_week = values[15:20].mean()
    month = []
    week = []
    for i in range (21,len (values)):
        month_mov = values[i-20:i].mean()
        week_mov = values[i-5:i].mean()  
        month.append(month_mov)
        week.append(week_mov)
        if pre_month > pre_week and month_mov <= week_mov:
            if abs(values[i-n]-values[i])/values[i-n] > r:
                result_true.append (i-n)
        elif pre_month < pre_week and month_mov >= week_mov:
            if abs (values [i-n]-values[i])/values[i-n] > r:
               result_true.append(i-n)
        elif abs (week_mov-month_mov)/min(week_mov, month_mov) < 0.01:
            result_false.append (i-n)
        pre_month = month_mov
        pre_week = week_mov
    return result_true, result_false



In [355]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = get_doc_by_date(up_date_train, news)
down_docs_train = get_doc_by_date(down_date_train, news)


In [356]:
len(up_docs_train)

1701

In [357]:
keyword_select(up_docs_train, '大立光')

['不理亞股下挫 歐股午盤上揚 不理亞股下挫 歐股午盤上揚 泛歐fteurofirst 指數和績優股euro stoxx指數分別上揚 和 泛歐fteurofirst 指數上週重挫 中國大陸股市暴跌引爆股災 蔓延至全球股市 中央社 翻譯 ',
 '本田civic和富豪suv 膺選北美年度風雲車 本田civic和富豪suv 膺選北美年度風雲車 法新社底特律日電 在 汽車與卡車雜誌 car truck magazine 今天舉行的頒獎典禮上 本田civic honda civic 和富豪休旅車xc volvo xc 分別膺選北美年度風雲汽車和風雲卡車 美國富豪汽車 volvo cars usa 執行長柯西梅克斯 lex kerrsemakers 表示 由名汽車記者遴選的此獎項 將有助於放眼重新躋身美國更大汽車品牌的富豪汽車 富豪隸屬中國大陸吉利汽車集團 geely 柯西梅克斯說 此款車輛結合富豪的超級安全信譽和更佳油耗 部分是拜採用缸引擎之賜 這顯示我們顯然走在正確途徑上 柯西梅克斯指出 中國大陸母公司賦予富豪獨立營運地位 本田執行副總裁蒙德爾 john mendel 表示 他對獲獎深感振奮 並稱 本田civic不會止步於此 ',
 '能源盤後 中國需求減緩 原油收低 美元 跌破每桶美元 能源盤後 中國需求減緩 原油收低 美元 跌破每桶美元 鉅亨網編譯郭照青美國原油期貨週一收盤跌破每桶美元 為自年月以來首度跌破該關卡 擔憂中國能源需求減緩 及預期伊朗很快將賣出原油 使得原油價格連續第六天走低 紐約月原油下跌 美元 或 收於每桶 美元 創年月日以來 最低收盤水準 上週 原油大跌逾 分析師說 紐約原油跌破美元 意義重大 因該關卡為主要技術性支撐 投資人將注意來自中國的訊息 與美國經濟數據 月布倫特原油下跌美元 或 收於每桶 美元 擔憂中國的經濟 持續對市場氣氛構成沉重壓力 分析師說 整體而言 錯綜的中國經濟與美國以外地區寬鬆的貨幣政策 是原油疲弱的重要原因 幾項不同的因素 促使美元走強 包括聯準會 fed 升息 增添了原油市場壓力 亦有分析師說 目前實際驅動能源市場的因素 是全球生產過多及接近記錄高點的庫存 在美國產量開始下降之前 油價可能進一步走低 短期內 原油可能跌破每桶美元 有跡象顯示 生產可能很快減緩 baker hughes公司公布 上週美國作業鑽油機數量減少台至台

In [358]:
len(keyword_select(up_docs_train, '大立光'))

1393

In [359]:
len(keyword_select(up_docs_train, ['大立光', '漲']))

661

In [360]:
len(keyword_select(up_docs_train, ['大立光']))

157

In [361]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','漲','股票'])
down_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','跌','股票'])


In [362]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = get_doc_by_date(up_date_test, news)
down_docs_test = get_doc_by_date(down_date_test, news)

# Request 3 - Total evaluation (find up down factor)

In [363]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


(1573, 1500)


In [364]:
test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
X_test = test_x.toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


(1922, 1500)


In [365]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[1153  217]
 [ 466   86]]
             precision    recall  f1-score   support

          0       0.71      0.84      0.77      1370
          1       0.28      0.16      0.20       552

avg / total       0.59      0.64      0.61      1922

0.6446409989594173


In [366]:
stock2016 = get_stock_data('2016_stock_data.csv', '大立光')
stock2017 = get_stock_data('2017_stock_data.csv', '大立光')
stock2018 = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
stock_train = np.concatenate((stock2016, stock2017), axis=0)
end_value_train = conv_num(stock_train[:, 5])
end_value2018 = conv_num(stock2018[:, 5])
stock_date_train = stock_train[:, 1]
stock_date2018 = stock2018[:, 1]


In [367]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','漲','股票'])
down_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','跌','股票'])


In [368]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = get_doc_by_date(up_date_test, news)
down_docs_test = get_doc_by_date(down_date_test, news)

# Request 3 - Total evaluation (find up down factor)

In [369]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


(2172, 1500)


In [370]:
test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
X_test = test_x.toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


(13854, 1500)


In [371]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[8193 4203]
 [ 866  592]]
             precision    recall  f1-score   support

          0       0.90      0.66      0.76     12396
          1       0.12      0.41      0.19      1458

avg / total       0.82      0.63      0.70     13854

0.6341128915836581


In [372]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','漲','股票'])
down_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','跌','股票'])

# Request 3 - Total evaluation (find up down factor)

In [373]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


(2172, 1500)


In [374]:
test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
X_test = test_x.toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


(1235, 1500)


In [375]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[334 249]
 [384 268]]
             precision    recall  f1-score   support

          0       0.47      0.57      0.51       583
          1       0.52      0.41      0.46       652

avg / total       0.49      0.49      0.48      1235

0.4874493927125506


In [376]:
# test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
temp = TfidfVectorizer(vocabulary = docs_vector.get_feature_names())
X_test = temp.fit_transform(up_docs_test+down_docs_test).toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


In [377]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[538  45]
 [584  68]]
             precision    recall  f1-score   support

          0       0.48      0.92      0.63       583
          1       0.60      0.10      0.18       652

avg / total       0.54      0.49      0.39      1235

0.49068825910931174


In [378]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


(2172, 1500)


In [379]:
# test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
temp = TfidfVectorizer(vocabulary = docs_vector.get_feature_names())
X_test = temp.fit_transform(up_docs_test+down_docs_test).toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


In [380]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[538  45]
 [584  68]]
             precision    recall  f1-score   support

          0       0.48      0.92      0.63       583
          1       0.60      0.10      0.18       652

avg / total       0.54      0.49      0.39      1235

0.49068825910931174


In [381]:
stock_train = get_stock_data('2016_stock_data.csv', '大立光')
stock2018 = get_stock_data('2017_stock_data.csv', '大立光')
# stock2018 = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
# stock_train = np.concatenate((stock2016, stock2017), axis=0)
end_value_train = conv_num(stock_train[:, 5])
end_value2018 = conv_num(stock2018[:, 5])
stock_date_train = stock_train[:, 1]
stock_date2018 = stock2018[:, 1]


In [382]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','漲','股票'])
down_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','跌','股票'])


In [383]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','漲','股票'])
down_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','跌','股票'])

# Request 3 - Total evaluation (find up down factor)

In [384]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


(1573, 1500)


In [385]:
# test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
temp = TfidfVectorizer(vocabulary = docs_vector.get_feature_names())
X_test = temp.fit_transform(up_docs_test+down_docs_test).toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


In [386]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[225   1]
 [258   3]]
             precision    recall  f1-score   support

          0       0.47      1.00      0.63       226
          1       0.75      0.01      0.02       261

avg / total       0.62      0.47      0.31       487

0.4681724845995893


# Requirement 1 - key word search

In [387]:
# stock_2016 = get_stock_data('2016_stock_data.csv', '大立光')
stock_train = get_stock_data('2017_stock_data.csv', '大立光')
stock2018 = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
# stock_train = np.concatenate((stock2016, stock2017), axis=0)
end_value_train = conv_num(stock_train[:, 5])
end_value2018 = conv_num(stock2018[:, 5])
stock_date_train = stock_train[:, 1]
stock_date2018 = stock2018[:, 1]


In [388]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','漲','股票'])
down_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','跌','股票'])


In [389]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','漲','股票'])
down_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','跌','股票'])

# Request 3 - Total evaluation (find up down factor)

In [390]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


(487, 1500)


In [391]:
# test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
temp = TfidfVectorizer(vocabulary = docs_vector.get_feature_names())
X_test = temp.fit_transform(up_docs_test+down_docs_test).toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


In [392]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[513  70]
 [572  80]]
             precision    recall  f1-score   support

          0       0.47      0.88      0.62       583
          1       0.53      0.12      0.20       652

avg / total       0.50      0.48      0.40      1235

0.4801619433198381


In [393]:
stock2018 = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
end_value2018 = conv_num(stock2018[:, 5])
stock_date2018 = stock2018[:, 1]


In [394]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','漲','股票'])
down_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','跌','股票'])

# Request 3 - Total evaluation (find up down factor)

In [395]:
# test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
temp = TfidfVectorizer(vocabulary = docs_vector.get_feature_names())
X = temp.fit_transform(up_docs_test+down_docs_test).toarray()
y = []
for i in range(len(up_docs_test)):
    y.append(1)
for i in range(len(down_docs_test)):
    y.append(0)
y = np.array(y)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X) 
np.random.seed(120)
np.random.shuffle(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [396]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[ 44 138]
 [102  87]]
             precision    recall  f1-score   support

          0       0.30      0.24      0.27       182
          1       0.39      0.46      0.42       189

avg / total       0.34      0.35      0.35       371

0.353099730458221


In [397]:
X_train

KeyboardInterrupt: 

In [398]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [399]:
y_train[0]

0

In [400]:
y_train

array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [401]:
len(y_train)

864

In [402]:
len(X)

1235

In [1]:
import re
import datetime
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


C:\Users\bioha\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
stopwords = []
with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
    for line in f1:
        stopwords = line.split()
stopwords[-1] = ' '

def get_company(company, stock_np, name_index = 0):
    indexs = []
    stock_np = np.array(stock_np)
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    return stock_np[indexs,:]

def get_stock_data(file, company_name = '大立光'):
    stock_data = pd.read_csv(file, encoding='UTF-8')
    stock_data_np = stock_data.values
    stock = get_company(company_name, stock_data_np)
    return stock[::-1,:]

def conv_num(nums):
    for i in range(len(nums)):
        nums[i] = nums[i].replace(',', '')
        nums[i] = float(nums[i])
    return nums

def conv_date(date):
    # date = np.array(date)
    for i in range(len(date)):
        d = re.findall('[0-9]+/[0-9]+/[0-9]+', date[i])[0]
        date[i] = datetime.datetime.strptime(date[i],'%Y/%M/%d').date()
        return date
    

def text_clean(text):
    text = text.lower()
    text = re.sub('<br>', ' ', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[^\w\s]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

def get_up_down_index(values, n = 5, r = 0.1):
    up = []
    down = []
	#後三天股價-原股價/原股價 ->漲跌幅度超過5%者
    for i in range(len(values) - n):
        dis  = values[i + n] - values[i]
        if abs(dis)/ values[i] >= r:
            up.append(i) if dis > 0 else down.append(i)
    return up, down

def get_doc_by_date(dates, data, date_index = 4, title_index = 5, content_index = 7):
    docs = []
    for r in data:
        if r[date_index][:-9] in dates:
            temp = '%s %s %s' %(r[title_index], r[title_index], r[content_index])
            docs.append(text_clean(temp))
    return docs

def keyword_select(docs, kewords):
    selected = []
    for text in docs:
        for i in range(len(kewords)):
            if kewords[i] in text:
                selected.append(text)
                break
    return selected

def tfidf_tool(corpus):
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords, max_features = 1500, max_df=0.8)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def get_top_feature(X, vectorizer):
    m = X.mean(axis=0).getA().reshape(-1)
    max_index = np.argsort(m)[::-1] 
    tokens = np.array(vectorizer.get_feature_names())
    return tokens[max_index]

def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words = stopwords)
    X = vectorizer.fit_transform(corpus)
    print(X.shape)
    return X, vectorizer

def chi2(all_vectors, clas_vectors):
    expected = all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    signs = np.where(clas_value-expected, 2)/expected
    chi2_value[signs] *= -1
    return chi2_value

def find_week_month_cross (values, n=3, r = 0.03):
    result_true = []
    result_false = []
    pre_month = values[:20].mean()
    pre_week = values[15:20].mean()
    month = []
    week = []
    for i in range (21,len (values)):
        month_mov = values[i-20:i].mean()
        week_mov = values[i-5:i].mean()  
        month.append(month_mov)
        week.append(week_mov)
        if pre_month > pre_week and month_mov <= week_mov:
            if abs(values[i-n]-values[i])/values[i-n] > r:
                result_true.append (i-n)
        elif pre_month < pre_week and month_mov >= week_mov:
            if abs (values [i-n]-values[i])/values[i-n] > r:
               result_true.append(i-n)
        elif abs (week_mov-month_mov)/min(week_mov, month_mov) < 0.01:
            result_false.append (i-n)
        pre_month = month_mov
        pre_week = week_mov
    return result_true, result_false



# Requirement 1 - key word search

In [3]:
stock_train = get_stock_data('2016_stock_data.csv', '大立光')
stock2018 = get_stock_data('2017_stock_data.csv', '大立光')
# stock2018 = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
# stock_train = np.concatenate((stock2016, stock2017), axis=0)
end_value_train = conv_num(stock_train[:, 5])
end_value2018 = conv_num(stock2018[:, 5])
stock_date_train = stock_train[:, 1]
stock_date2018 = stock2018[:, 1]


In [4]:
up_train, down_train = get_up_down_index(end_value_train)
up_date_train = stock_date_train[up_train]
down_date_train = stock_date_train[down_train]

up_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','漲','股票'])
down_docs_train = keyword_select(get_doc_by_date(up_date_train, news), ['大立光','跌','股票'])


In [5]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

up_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','漲','股票'])
down_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','跌','股票'])

# Request 3 - Total evaluation (find up down factor)

In [6]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


Building prefix dict from the default dictionary ...
Loading model from cache D:\Temp\jieba.cache
Loading model cost 1.024 seconds.
Prefix dict has been built succesfully.


(1573, 1500)


In [7]:
# test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
temp = TfidfVectorizer(vocabulary = docs_vector.get_feature_names())
X_test = temp.fit_transform(up_docs_test+down_docs_test).toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


In [8]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[225   1]
 [258   3]]
             precision    recall  f1-score   support

          0       0.47      1.00      0.63       226
          1       0.75      0.01      0.02       261

avg / total       0.62      0.47      0.31       487

0.4681724845995893


In [9]:
len(X)

NameError: name 'X' is not defined

In [10]:
len(docs_X)

NameError: name 'docs_X' is not defined

In [11]:
len(docs_x)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [12]:
docs_x.shape

(1573, 1500)

In [13]:
up_test, down_test = get_up_down_index(end_value2018)
up_date_test = stock_date2018[up_test]
down_date_test = stock_date2018[down_test]

# up_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','漲','股票'])
# down_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','跌','股票'])

# Request 3 - Total evaluation (find up down factor)

In [14]:
docs_x, docs_vector = tfidf_tool(up_docs_train+down_docs_train)
X_train = docs_x.toarray()
y_train = []
for i in range(len(up_docs_train)):
    y_train.append(1)
for i in range(len(down_docs_train)):
    y_train.append(0)
y_train = np.array(y_train)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(100)
np.random.shuffle(X_train) 
np.random.seed(100)
np.random.shuffle(y_train)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


(1573, 1500)


In [15]:
# test_x, test_vector = tfidf_tool(up_docs_test+down_docs_test)
temp = TfidfVectorizer(vocabulary = docs_vector.get_feature_names())
X_test = temp.fit_transform(up_docs_test+down_docs_test).toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
#===============shuffle data to make random state & split into training part and testing part========================
np.random.seed(120)
np.random.shuffle(X_test) 
np.random.seed(120)
np.random.shuffle(y_test)


In [16]:
#======================= model training ====================================
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[225   1]
 [258   3]]
             precision    recall  f1-score   support

          0       0.47      1.00      0.63       226
          1       0.75      0.01      0.02       261

avg / total       0.62      0.47      0.31       487

0.4681724845995893


In [17]:
X_test.shape

(487, 1500)

In [18]:
len(y_test)

487

In [19]:
len(y_train)

1573

# Requirement 1 - key word search

In [20]:
stock = get_stock_data('2018_stock_data.csv', '大立光')
news = pd.read_csv('news.csv', encoding= 'big5')
news = np.array(news)
end_value = conv_num(stock[:, 5])
stock_date = stock[:, 1]


In [21]:
up, down = get_up_down_index(end_value)
up_date = stock_date[up]
down_date = stock_date[down]

up_docs_test = keyword_select(get_doc_by_date(up_date, news), ['大立光','漲','股票'])
down_docs = up_docs_test = keyword_select(get_doc_by_date(down_date, news), ['大立光','跌','股票'])


# Request 3 - Total evaluation (find up down factor)

In [22]:
docs_x, docs_vector = tfidf_tool(up_docs+down_docs)
X = docs_x.toarray()
# X = np.concatenate((up_x, down_x), axis=0)
Y = []
for i in range(len(up_x)):
    Y.append(1)
for i in range(len(down_x)):
    Y.append(0)
Y = np.array(Y)

NameError: name 'up_docs' is not defined

In [23]:
np.random.seed(100)
np.random.shuffle(X) 
np.random.seed(100)
np.random.shuffle(Y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


NameError: name 'X' is not defined

In [24]:
#======================= model training ====================================
from sklearn.ensemble import RandomForestClassifier
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[225   1]
 [258   3]]
             precision    recall  f1-score   support

          0       0.47      1.00      0.63       226
          1       0.75      0.01      0.02       261

avg / total       0.62      0.47      0.31       487

0.4681724845995893


In [25]:
up, down = get_up_down_index(end_value)
up_date = stock_date[up]
down_date = stock_date[down]

up_docs = keyword_select(get_doc_by_date(up_date, news), ['大立光','漲','股票'])
down_docs = up_docs_test = keyword_select(get_doc_by_date(down_date, news), ['大立光','跌','股票'])


# Request 3 - Total evaluation (find up down factor)

In [26]:
docs_x, docs_vector = tfidf_tool(up_docs+down_docs)
X = docs_x.toarray()
# X = np.concatenate((up_x, down_x), axis=0)
Y = []
for i in range(len(up_x)):
    Y.append(1)
for i in range(len(down_x)):
    Y.append(0)
Y = np.array(Y)

(4492, 1500)


NameError: name 'up_x' is not defined

In [27]:
np.random.seed(100)
np.random.shuffle(X) 
np.random.seed(100)
np.random.shuffle(Y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


ValueError: Found input variables with inconsistent numbers of samples: [4492, 0]

In [28]:
#======================= model training ====================================
from sklearn.ensemble import RandomForestClassifier
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[225   1]
 [258   3]]
             precision    recall  f1-score   support

          0       0.47      1.00      0.63       226
          1       0.75      0.01      0.02       261

avg / total       0.62      0.47      0.31       487

0.4681724845995893


In [29]:
docs_x, docs_vector = tfidf_tool(up_docs+down_docs)
X = docs_x.toarray()
# X = np.concatenate((up_x, down_x), axis=0)
Y = []
for i in range(len(up_docs)):
    Y.append(1)
for i in range(len(down_docs)):
    Y.append(0)
Y = np.array(Y)

(4492, 1500)


In [30]:
np.random.seed(100)
np.random.shuffle(X) 
np.random.seed(100)
np.random.shuffle(Y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


In [31]:
#======================= model training ====================================
from sklearn.ensemble import RandomForestClassifier
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
classifier1 = RandomForestClassifier(n_estimators=5, random_state=1)  
classifier1.fit(X_train, y_train)  
y_pred = classifier1.predict(X_test) 
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))




# #%%[markdown]
# ## Requirement 2 - cross search

# #%%
# sigdif, indif = find_week_month_cross(end_value)
# sigdif_date = stock_date[sigdif]
# indif_date = stock_date[indif]
# sigdif_docs = get_doc_by_date(sigdif_date, news)
# indif_docs = get_doc_by_date(indif_date, news)

# #%%
# cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
# X = cross_x.toarray()
# # X = np.concatenate((sig_x, indif_x), axis=0)
# Y = []
# for i in range(len(sig_x)):
#     Y.append(1)
# for i in range(len(indif_x)):
#     Y.append(0)
# Y = np.array(Y)
# np.random.seed(100)
# np.random.shuffle(X) 
# np.random.seed(100)
# np.random.shuffle(Y)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
# #%%
# #%%
# #======================= model training ====================================
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
# classifier2 = RandomForestClassifier(n_estimators=5, random_state=0)  
# classifier2.fit(X_train, y_train)  
# y_pred = classifier2.predict(X_test) 
# #====================== model evaluation ======================================
# print(confusion_matrix(y_test,y_pred))  
# print(classification_report(y_test,y_pred))  
# print(accuracy_score(y_test, y_pred))


[[1126   52]
 [ 116   54]]
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      1178
          1       0.51      0.32      0.39       170

avg / total       0.86      0.88      0.86      1348

0.8753709198813057


In [32]:
len(y_train)

3144

In [33]:
len(y_test)

1348

# Requirement 2 - cross search

In [34]:
sigdif, indif = find_week_month_cross(end_value)
sigdif_date = stock_date[sigdif]
indif_date = stock_date[indif]
sigdif_docs = get_doc_by_date(sigdif_date, news)
indif_docs = get_doc_by_date(indif_date, news)


In [35]:
cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
X = cross_x.toarray()
# X = np.concatenate((sig_x, indif_x), axis=0)
Y = []
for i in range(len(sig_x)):
    Y.append(1)
for i in range(len(indif_x)):
    Y.append(0)
Y = np.array(Y)
np.random.seed(100)
np.random.shuffle(X) 
np.random.seed(100)
np.random.shuffle(Y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

(21142, 1500)


NameError: name 'sig_x' is not defined

In [37]:
#======================= model training ====================================
cross_classifier = RandomForestClassifier(n_estimators=5, random_state=0)  
cross_classifier.fit(X_train, y_train)  
y_pred = cross_classifier.predict(X_test) 
#====================== model evaluation ======================================
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))


[[1126   52]
 [ 124   46]]
             precision    recall  f1-score   support

          0       0.90      0.96      0.93      1178
          1       0.47      0.27      0.34       170

avg / total       0.85      0.87      0.85      1348

0.8694362017804155


In [38]:
cross_x, cross_vector = tfidf_tool(sigdif_docs + indif_docs)
X = cross_x.toarray()
# X = np.concatenate((sig_x, indif_x), axis=0)
Y = []
for i in range(len(sigdif_docs)):
    Y.append(1)
for i in range(len(indif_docs)):
    Y.append(0)
Y = np.array(Y)
np.random.seed(100)
np.random.shuffle(X) 
np.random.seed(100)
np.random.shuffle(Y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

(21142, 1500)


In [39]:
#======================= model training ====================================
cross_classifier = RandomForestClassifier(n_estimators=5, random_state=0)  
cross_classifier.fit(X_train, y_train)  
y_pred = cross_classifier.predict(X_test) 
#====================== model evaluation ======================================
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))


[[4280  538]
 [1079  446]]
             precision    recall  f1-score   support

          0       0.80      0.89      0.84      4818
          1       0.45      0.29      0.36      1525

avg / total       0.72      0.75      0.72      6343

0.7450733091597036


In [40]:
#======================= model training ====================================
cross_classifier = RandomForestClassifier(n_estimators=10, random_state=0)  
cross_classifier.fit(X_train, y_train)  
y_pred = cross_classifier.predict(X_test) 
#====================== model evaluation ======================================
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))


[[4486  332]
 [1154  371]]
             precision    recall  f1-score   support

          0       0.80      0.93      0.86      4818
          1       0.53      0.24      0.33      1525

avg / total       0.73      0.77      0.73      6343

0.7657259971622261


In [41]:
up, down = get_up_down_index(end_value)
up_date = stock_date[up]
down_date = stock_date[down]
# select important keyword to filter documents
up_docs = keyword_select(get_doc_by_date(up_date, news), ['大立光','漲','股票'])
down_docs = keyword_select(get_doc_by_date(down_date, news), ['大立光','跌','股票'])


# Request 3 - Total evaluation (find up down factor)

In [42]:
docs_x, docs_vector = tfidf_tool(up_docs+down_docs)
X = docs_x.toarray()
# X = np.concatenate((up_x, down_x), axis=0)
Y = []
for i in range(len(up_docs)):
    Y.append(1)
for i in range(len(down_docs)):
    Y.append(0)
Y = np.array(Y)

(4492, 1500)


In [43]:
np.random.seed(100)
np.random.shuffle(X) 
np.random.seed(100)
np.random.shuffle(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

#%%
#======================= model training ====================================
from sklearn.ensemble import RandomForestClassifier
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
up_down_classifier = RandomForestClassifier(n_estimators=5, random_state=1)  
up_down_classifier.fit(X_train, y_train)  
y_pred = up_down_classifier.predict(X_test)
#====================== model evaluation ======================================
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1126   52]
 [ 116   54]]
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      1178
          1       0.51      0.32      0.39       170

avg / total       0.86      0.88      0.86      1348

0.8753709198813057


In [44]:
stock_test = get_stock_data('2017_stock_data.csv', '大立光')
end_value_test = conv_num(stock_test[:, 5])
stock_date_test = stock_test[:, 1]
up_test, down_test = get_up_down_index(end_value_test)
up_date_test = stock_date[up_test]
down_date_test = stock_date[down_test]
# select important keyword to filter documents
up_docs_test = keyword_select(get_doc_by_date(up_date_test, news), ['大立光','漲','股票'])
down_docs_test = keyword_select(get_doc_by_date(down_date_test, news), ['大立光','跌','股票'])

temp = TfidfVectorizer(vocabulary = docs_vector.get_feature_names())
X_test = temp.fit_transform(up_docs_test+down_docs_test).toarray()
y_test = []
for i in range(len(up_docs_test)):
    y_test.append(1)
for i in range(len(down_docs_test)):
    y_test.append(0)
y_test = np.array(y_test)
np.random.seed(150)
np.random.shuffle(X_test) 
np.random.seed(150)
np.random.shuffle(y_test)


In [45]:
y_pred = up_down_classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))









[[2476    5]
 [ 906    0]]
             precision    recall  f1-score   support

          0       0.73      1.00      0.84      2481
          1       0.00      0.00      0.00       906

avg / total       0.54      0.73      0.62      3387

0.7310304103926779


# Requirement 2 - cross search
use function find_week_month_cross(end_value) to get the index that satisfied our expectation
data have two class: 1. the moving average that has significant change or 2.the moving average that is indifferent
get the date of both two and find the document in those two days to train the predict model

# Requirement 2 - cross search
use function find_week_month_cross(end_value) to get the index that satisfied our expectation\n
data have two class: 1. the moving average that has significant change or 2.the moving average that is indifferent\n
get the date of both two and find the document in those two days to train the predict model\n